# Symbolic Math with SymPy function call example

Demonstrates how to delegate to [SymPy](https://www.sympy.org/en/index.html) to perform symbolic math. We use LaTeX to represent the math expressions and the [latex2sympy2](https://pypi.org/project/latex2sympy2/) to convert the LaTeX to SymPy expressions.

In [1]:
import openai
from latex2sympy2 import latex2sympy
import json

#model = "gpt-3.5-turbo-0613"
model = "gpt-4-0613"

def calc_latex_math_expression(expression):
    sympy_expr = latex2sympy(expression)
    result = sympy_expr.doit()
    return str(result)

functions = [
     {
      "name": "calc_latex_math_expression",
      "description": "Calculate a Latex mathematical expression",
      "parameters": {
        "type": "object",
        "properties": {
          "expression": {
            "type": "string",
            "description": "The Latex expression to calculate"
          },
        },
        "required": ["expression"]
      }
    } 
  ]

available_functions = {
    "calc_latex_math_expression": calc_latex_math_expression,
}

In [2]:
def run_conversation(content):
  messages = [
    {"role": "user", "content": content},
    ]
  # Step 1: send the conversation and available functions to GPT
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions
  )
  response_message = response["choices"][0]["message"]
  messages.append(response_message)  # extend conversation with assistant's reply

  # Step 2: check if GPT wanted to call a function
  while response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    function_name = response_message["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)

    # Step 4: send the info on the function call and function response to GPT
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    response_message = response["choices"][0]["message"]
    messages.append(response_message)  # extend conversation with assistant's reply

  for message in messages:
    if message.get("content"):
      print(message["role"] + ": " + message["content"])
    if message.get("function_call"):
      print(" -> calling function " + message["function_call"]["name"] + " with arguments " + json.dumps(message["function_call"]["arguments"]))
  

In [3]:
run_conversation("If I am 42 years old and my brother is 3 years younger than me, how old is my brother? Explain step by step.")

user: If I am 42 years old and my brother is 3 years younger than me, how old is my brother? Explain step by step.
assistant: Sure, let's solve this step by step:

1. Express the situation mathematically: You're 42 years old, and your brother is 3 years younger than you. This can be represented with the equation: Your age - 3 = Your brother's age.

2. In this equation, your age is given as 42, so we substitute it in: 42 - 3 = Your brother's age.

3. To find out your brother's age, we calculate the expression 42 - 3.

Let's perform the calculation using a function.
 -> calling function calc_latex_math_expression with arguments "{\n\"expression\": \"42 - 3\"\n}"
function: 39
assistant: 4. So, the result of 42 - 3 is 39.

Therefore, if you're 42 years old and your brother is 3 years younger than you, your brother is 39 years old.


In [4]:
run_conversation("If a circle has a radius of 3 cm, what is its symbolic area? Explain step by step.")

user: If a circle has a radius of 3 cm, what is its symbolic area? Explain step by step.
assistant: The area of a circle is given by the formula:

A = πr²

where:
- A is the area of the circle
- r is the radius of the circle
- π (Pi) is a constant approximately equal to 3.14159

In this case, the radius (r) is given as 3 cm. So, we plug that into our equation, we get the symbolic area of the circle as:

A = π(3cm)²

This is the symbolic representation of the area of the circle. It means we are squaring the radius (which is 3 cm) and then multiplying the result by Pi to get the area.

The calculation could proceed as follows: 

(3cm)² = 9 cm² (because 3 cm * 3 cm = 9 cm²), 

and then π(9 cm²) would give us the exact area in cm². 

To get the numerical result of the computation according to the Pi value, we can calculate this.
 -> calling function calc_latex_math_expression with arguments "{\n\"expression\": \"\\\\pi * 3^2\"\n}"
function: 9*pi
assistant: The exact symbolic area of the ci

In [5]:
run_conversation("If a sphere has a radius of 3 cm, what is its volume? Explain step by step.")

user: If a sphere has a radius of 3 cm, what is its volume? Explain step by step.
assistant: The formula to calculate the volume of a sphere is:

V = 4/3 * π * r³

Where V is the volume, r is the radius of the sphere, and π is approximately 3.14159.

Since the problem states the radius is 3 cm, we simply insert 3 cm into the formula in place of r:

V = 4/3 * π * (3 cm)³

I will do the calculation now.
 -> calling function calc_latex_math_expression with arguments "{\n  \"expression\": \"4/3 * 3.14159 * 3^3\"\n}"
function: 2827431/25000
assistant: So the volume of the sphere with a radius of 3 cm is 113.097 cm³.
